In [1]:
import numpy as np
import pandas as pd

In [8]:
datafile='https://raw.githubusercontent.com/evertongago/hadoop/master/ml-100k/u.data'
data=pd.read_csv(datafile,sep='\t',header=None,names=['userID','itemID','rating','timestamp'])

In [9]:
data.head(10)

,userID,itemID,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [15]:
movieInfoFile='https://raw.githubusercontent.com/evertongago/hadoop/master/ml-100k/u.item'
movieinfo=pd.read_csv(movieInfoFile,sep='|',header=None,
                      index_col=False,names=['itemId','title'],usecols=[0,1],encoding='latin-1')


In [16]:
movieinfo.head()

,itemId,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [17]:
data=pd.merge(data,movieinfo,left_on='itemID',right_on='itemId')

In [18]:
data.head()

,userID,itemID,rating,timestamp,itemId,title
0,196,242,3,881250949,242,Kolya (1996)
1,63,242,3,875747190,242,Kolya (1996)
2,226,242,5,883888671,242,Kolya (1996)
3,154,242,3,879138235,242,Kolya (1996)
4,306,242,5,876503793,242,Kolya (1996)


In [22]:
def favoriteMovies(activeUser,N):
    topMovies=pd.DataFrame.sort_values(data[data.userID==activeUser],['rating'],ascending=[0])[:N]
    return list(topMovies.title)


#data[data.userID==activeUser]

In [23]:
print(favoriteMovies(5,3))

['Empire Strikes Back, The (1980)', 'Blade Runner (1982)', 'Wrong Trousers, The (1993)']


In [29]:
userItemRatingMatrix=pd.pivot_table(data,values='rating',index=['userID'],columns=['itemID'])

In [30]:
userItemRatingMatrix.head()

itemID,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userID,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
from scipy.spatial.distance import correlation
def similarity(user1,user2):
    user1=np.array(user1)-np.nanmean(user1)
    user2=np.array(user2)-np.nanmean(user2)
    commonItemIds=[i for i in range(len(user1)) if user1[i]>0 and user2[i]>0]
    
    if len(commonItemIds)==0:
        return 0
    else:
        user1=np.array([user1[i] for i in commonItemIds])
        user2=np.array([user2[i] for i in commonItemIds])
        return correlation(user1,user2)

In [32]:
def nearestNeighbourRatings(activeUser,K):
    # This function will find the K Nearest neighbours of the active user, then 
    # use their ratings to predict the activeUsers ratings for other movies 
    similarityMatrix=pd.DataFrame(index=userItemRatingMatrix.index,
                                  columns=['Similarity'])
    # Creates an empty matrix whose row index is userIds, and the value will be 
    # similarity of that user to the active User
    for i in userItemRatingMatrix.index:
        similarityMatrix.loc[i]=similarity(userItemRatingMatrix.loc[activeUser],
                                          userItemRatingMatrix.loc[i])
        # Find the similarity between user i and the active user and add it to the 
        # similarityMatrix 
    similarityMatrix=pd.DataFrame.sort_values(similarityMatrix,
                                              ['Similarity'],ascending=[0])
    # Sort the similarity matrix in the descending order of similarity 
    nearestNeighbours=similarityMatrix[:K]
    # The above line will give us the K Nearest neighbours 
    
    # We'll now take the nearest neighbours and use their ratings 
    # to predict the active user's rating for every movie
    neighbourItemRatings=userItemRatingMatrix.loc[nearestNeighbours.index]
    # There's something clever we've done here
    # the similarity matrix had an index which was the userId, By sorting 
    # and picking the top K rows, the nearestNeighbours dataframe now has 
    # a dataframe whose row index is the userIds of the K Nearest neighbours 
    # Using this index we can directly find the corresponding rows in the 
    # user Item rating matrix 
    predictItemRating=pd.DataFrame(index=userItemRatingMatrix.columns, columns=['Rating'])
    # A placeholder for the predicted item ratings. It's row index is the 
    # list of itemIds which is the same as the column index of userItemRatingMatrix
    #Let's fill this up now
    for i in userItemRatingMatrix.columns:
        # for each item 
        predictedRating=np.nanmean(userItemRatingMatrix.loc[activeUser])
        # start with the average rating of the user
        for j in neighbourItemRatings.index:
            # for each neighbour in the neighbour list 
            if userItemRatingMatrix.loc[j,i]>0:
                # If the neighbour has rated that item
                # Add the rating of the neighbour for that item
                #    adjusted by 
                #    the average rating of the neighbour 
                #    weighted by 
                #    the similarity of the neighbour to the active user
                predictedRating += (userItemRatingMatrix.loc[j,i]
                                    -np.nanmean(userItemRatingMatrix.loc[j]))*nearestNeighbours.loc[j,'Similarity']
        # We are out of the loop which uses the nearest neighbours, add the 
        # rating to the predicted Rating matrix
        predictItemRating.loc[i,'Rating']=predictedRating
    return predictItemRating


        

In [35]:
def topNRecommendations(activeUser,N):
    predictItemRating=nearestNeighbourRatings(activeUser,10)
    # Use the 10 nearest neighbours to find the predicted ratings
    moviesAlreadyWatched=list(userItemRatingMatrix.loc[activeUser]
                              .loc[userItemRatingMatrix.loc[activeUser]>0].index)
    # find the list of items whose ratings which are not NaN
    predictItemRating=predictItemRating.drop(moviesAlreadyWatched)
    topRecommendations=pd.DataFrame.sort_values(predictItemRating,
                                                ['Rating'],ascending=[0])[:N]
    # This will give us the list of itemIds which are the top recommendations 
    # Let's find the corresponding movie titles 
    topRecommendationTitles=(movieinfo.loc[movieinfo.itemId.isin(topRecommendations.index)])
    return list(topRecommendationTitles.title)


In [36]:
activeUser=5
print(favoriteMovies(activeUser,5),"\n",topNRecommendations(activeUser,3))


C:\Users\venka\Anaconda3\lib\site-packages\scipy\spatial\distance.py:543: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(um, vm) / (norm(um) * norm(vm))


['Empire Strikes Back, The (1980)', 'Blade Runner (1982)', 'Wrong Trousers, The (1993)', 'Duck Soup (1933)', 'Return of the Pink Panther, The (1974)'] 
 ['Truth About Cats & Dogs, The (1996)', 'Scream (1996)', 'First Wives Club, The (1996)']


In [37]:
# In[30]:

# Let's now use matrix factorization to do the same exercise ie
# finding the recommendations for a user
# The idea here is to identify some factors (these are factors which influence
# a user'r rating). The factors are identified by decomposing the 
# user item rating matrix into a user-factor matrix and a item-factor matrix
# Each row in the user-factor matrix maps the user onto the hidden factors
# Each row in the product factor matrix maps the item onto the hidden factors
# This operation will be pretty expensive because it will effectively give us 
# the factor vectors needed to find the rating of any product by any user 
# (in the  previous case we only did the computations for 1 user)

def matrixFactorization(R, K, steps=10, gamma=0.001,lamda=0.02):
    # R is the user item rating matrix 
    # K is the number of factors we will find 
    # We'll be using Stochastic Gradient descent to find the factor vectors 
    # steps, gamma and lamda are parameters the SGD will use - we'll get to them
    # in a bit 
    N=len(R.index)# Number of users
    M=len(R.columns) # Number of items 
    P=pd.DataFrame(np.random.rand(N,K),index=R.index)
    # This is the user factor matrix we want to find. It will have N rows 
    # on for each user and K columns, one for each factor. We are initializing 
    # this matrix with some random numbers, then we will iteratively move towards 
    # the actual value we want to find 
    Q=pd.DataFrame(np.random.rand(M,K),index=R.columns)
    # This is the product factor matrix we want to find. It will have M rows, 
    # one for each product/item/movie. 
    for step in range(steps):
        # SGD will loop through the ratings in the user item rating matrix 
        # It will do this as many times as we specify (number of steps) or 
        # until the error we are minimizing reaches a certain threshold 
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    # For each rating that exists in the training set 
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    # This is the error for one rating 
                    # ie difference between the actual value of the rating 
                    # and the predicted value (dot product of the corresponding 
                    # user factor vector and item-factor vector)
                    # We have an error function to minimize. 
                    # The Ps and Qs should be moved in the downward direction 
                    # of the slope of the error at the current point 
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    # Gamma is the size of the step we are taking / moving the value
                    # of P by 
                    # The value in the brackets is the partial derivative of the 
                    # error function ie the slope. Lamda is the value of the 
                    # regularization parameter which penalizes the model for the 
                    # number of factors we are finding. 
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        # At the end of this we have looped through all the ratings once. 
        # Let's check the value of the error function to see if we have reached 
        # the threshold at which we want to stop, else we will repeat the process
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    #Sum of squares of the errors in the rating
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        print(step)
    return P,Q

# Let's call this function now 
(P,Q)=matrixFactorization(userItemRatingMatrix.iloc[:100,:100],K=2,gamma=0.001,lamda=0.02, steps=100)
# Ideally you should run this over the entire matrix for a few 1000 steps, 
# This will be pretty expensive computationally. For now lets just do it over a 
# part of the rating matrix to see how it works. We've kept the steps to 100 too. 


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [39]:
activeUser=1
predictItemRating=pd.DataFrame(np.dot(P.loc[activeUser],Q.T),index=Q.index,columns=['Rating'])
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:3]
# We found the ratings of all movies by the active user and then sorted them to find the top 3 movies 
topRecommendationTitles=movieinfo.loc[movieinfo.itemId.isin(topRecommendations.index)]
print(list(topRecommendationTitles.title))


['Star Wars (1977)', 'Shawshank Redemption, The (1994)', 'Fargo (1996)']


In [40]:

import itertools 
# This module will help us generate all permutations of movies
# We'll use that to find the possible rules and then filter for those with 
# the required confidence

allitems=[]

def support(itemset):
    userList=userItemRatingMatrix.index
    nUsers=len(userList)
    ratingMatrix=userItemRatingMatrix
    for item in itemset:
        ratingMatrix=ratingMatrix.loc[ratingMatrix.loc[:,item]>0]
        #Subset the ratingMatrix to the set of users who have rated this item 
        userList=ratingMatrix.index
    # After looping through all the items in the set, we are left only with the
    # users who have rated all the items in the itemset
    return float(len(userList))/float(nUsers)
# Support is the proportion of all users who have watched this set of movies 

minsupport=0.3
for item in list(userItemRatingMatrix.columns):
    itemset=[item]
    if support(itemset)>minsupport:
        allitems.append(item)
# We are now left only with the items which have been rated by atleast 30% of 
#the users
        




# In[36]:

len(allitems)


# In[37]:

# 47 of the movies were watched by atleast 30% of the users. From these movies
# we'll generate rules and test again for support and confidence
minconfidence=0.1
assocRules=[]
i=2
for rule in itertools.permutations(allitems,2):
    #Generates all possible permutations of 2 items from the remaining
    # list of 47 movies 
    from_item=[rule[0]]
    to_item=rule
    # each rule is a tuple of 2 items 
    confidence=support(to_item)/support(from_item)
    if confidence>minconfidence and support(to_item)>minsupport:
        assocRules.append(rule)


# In[38]:

# This will generate all possible 2 item rules which satisfy the support and 
# confidence constraints. 
# You can continue on and write a similar bit of code for finding 3 item rules 
# or even n item rules. At each step make sure that every rule satisfies minconfidence
# and minsupport


# In[39]:

assocRules



[(1, 50),
 (1, 100),
 (1, 117),
 (1, 121),
 (1, 181),
 (7, 50),
 (7, 100),
 (7, 181),
 (50, 1),
 (50, 7),
 (50, 56),
 (50, 69),
 (50, 79),
 (50, 98),
 (50, 100),
 (50, 117),
 (50, 121),
 (50, 127),
 (50, 172),
 (50, 173),
 (50, 174),
 (50, 181),
 (50, 204),
 (50, 210),
 (50, 222),
 (50, 237),
 (50, 258),
 (50, 288),
 (50, 294),
 (50, 405),
 (56, 50),
 (56, 98),
 (56, 100),
 (56, 174),
 (56, 181),
 (69, 50),
 (79, 50),
 (79, 174),
 (98, 50),
 (98, 56),
 (98, 100),
 (98, 174),
 (98, 181),
 (100, 1),
 (100, 7),
 (100, 50),
 (100, 56),
 (100, 98),
 (100, 117),
 (100, 121),
 (100, 127),
 (100, 174),
 (100, 181),
 (100, 237),
 (117, 1),
 (117, 50),
 (117, 100),
 (117, 121),
 (117, 181),
 (121, 1),
 (121, 50),
 (121, 100),
 (121, 117),
 (121, 181),
 (121, 405),
 (127, 50),
 (127, 100),
 (127, 181),
 (172, 50),
 (172, 174),
 (172, 181),
 (173, 50),
 (174, 50),
 (174, 56),
 (174, 79),
 (174, 98),
 (174, 100),
 (174, 172),
 (174, 181),
 (174, 204),
 (174, 210),
 (181, 1),
 (181, 7),
 (181, 50),
